# Analyze an image stack

In [ ]:
import astro3d
from astro3d import image_stack
import matplotlib.pyplot as plt

## Read the image stack

We again need to set the printer-specific settings and the directory of the slides:

Note: original print was: `LocalBox``

In [ ]:
stack = image_stack.IStack(astro3d.get_output('LocalBox'))

In [ ]:
stack = image_stack.IStack(astro3d.get_output('LocalTry6_redo'))

## Show view

In [ ]:
stack.show_colors()

Now we assign the transparent color index

In [ ]:
stack.empty_indices = [1]

In [ ]:
stack.show_view('xy', backward=True, bg=(127, 127, 127))

**Note:** as we replace one color with an existing one, there will be less colors afterwards, so we need to check the indices carefully. First replacement is similar, just the first index is changeing color. Second change effectively removes a color, so the later colors move up in index.

In [ ]:
%%time
stack.replace_color(2, image_stack.VeroM_sRGB)
stack.replace_color(3, image_stack.VeroC_sRGB)

In [ ]:
stack.show_colors()

In [ ]:
stack.empty_indices = [1]

In [ ]:
import matplotlib.pyplot as plt
f, ax = plt.subplots(dpi=400)
f, ax = stack.show_view('xy', backward=True, ax=ax)

In [ ]:
f, ax = stack.three_views()
f.dpi = 200

### Add logos

In [ ]:
sep = 0.3 # separation between logos and edges in cm
height = 1.125 # height of the logos
depth = 0.09 # how thick the logos will be

As perfect black is used in the image, we will use that black also in our palette

In [ ]:
pal = image_stack.vero_palette.copy()
pal[0] = [0, 0, 0]

#### Origin Logo

Here we have to add an alpha channel to make the white parts transparent

In [ ]:
from pkg_resources import resource_filename
from pathlib import Path
from PIL import Image

logo = Path(resource_filename('astro3d', '')).parent / 'figures' / 'Logo_Origins2.png'
logo_shape = np.array(Image.open(logo)).shape
width_o = height / logo_shape[0] * logo_shape[1]

Add origins logo using dithering of Cyan, Magenta, White

In [ ]:
stack.add_logo(logo, [sep, sep, sep], None, depth, height=height, plane='xz', threshold=0.8,
               pal=[pal[2], [128, 128, 128]])

Check the colors: now the white material appears first, we need to update that gray remains transparent

Now we replace red with magenta and blue with cyan

In [ ]:
stack.replace_color(4, stack.colors[3])
stack.replace_color(4, stack.colors[2])

In [ ]:
stack.show_colors()

In [ ]:
stack.empty_indices = [1]

In [ ]:
f, ax = plt.subplots(dpi=400)
f, ax = stack.show_view('xy', backward=True, ax=ax, bg=(0, 0, 0))

In [ ]:
f, ax = plt.subplots(dpi=400)
f, ax = stack.show_view('xz', ax=ax, bg=(0, 0, 0))

#### Localize Logo

In [ ]:
import requests
from io import BytesIO

response = requests.get('https://localization.ias.universite-paris-saclay.fr/sites/localization/files/Logo_Localization_150.png')
img = Image.open(BytesIO(response.content))

We dither this with the base palette. Here's what it will approximately look like (this is unscaled to voxel sizes).

In [ ]:
img_d = image_stack.dither_palette(img, pal)
plt.imshow(img_d);

In [ ]:
stack.add_logo(img, [2 * sep + width_o, sep, sep], None,
               depth, height=height, plane='xz', pal=pal)

In [ ]:
stack.show_colors()

In [ ]:
stack.empty_indices = [2]

In [ ]:
f, ax = stack.three_views()
f.dpi = 200

### Save Stack

In [ ]:
stack.show_colors(titles=['White', 'Yellow', 'UltraClear', 'Cyan', 'Magenta', 'Black']);

In [ ]:
stack.dimension

In [ ]:
out_dir = stack.directory.parent / (stack.directory.stem + '_logos')
stack.save_images(out_dir)
!zip -r {out_dir} {out_dir}
!mv {out_dir}.zip /Users/birnstiel/nextcloud/shared/3d-image-stacks/
!open /Users/birnstiel/nextcloud/shared/3d-image-stacks/